In [2]:
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.parallel import DistributedDataParallel

## Prepare Data for Deep Learning Model

In [7]:
# Load data
data_file_path = 'data/exercise_recognition_data.csv'
df = pd.read_csv(data_file_path)
df = df.drop(columns=['Unnamed: 0'])
df.head()

,set_num,activity,time,x_gyro,y_gyro,z_gyro,x_acc,y_acc,z_acc,label
0,0,Band Pull-Down Row,676.115485,-15.147634,23.954241,14.705304,0.356986,-0.823410,-0.031189,46.0
1,0,Band Pull-Down Row,679.055465,-148.993389,-1.727050,-8.243589,0.019032,-0.887622,-0.215872,46.0
2,0,Band Pull-Down Row,679.615462,147.988473,3.768917,95.452735,0.557560,-1.130520,0.688100,46.0
3,0,Band Pull-Down Row,681.575448,-140.093738,-29.372863,32.133631,0.448862,-0.735180,0.236496,46.0
4,0,Band Pull-Down Row,684.775427,-21.686398,-1.344477,-0.517016,0.367619,-0.557327,0.755909,46.0


In [8]:
# Split the data into features (X) and target (y)
X = df[['set_num', 'time', 'x_gyro', 'y_gyro', 'z_gyro', 'x_acc', 'y_acc', 'z_acc']]
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Set up GPU
Follow instruction to set up gpu5: https://medium.com/@mromerocalvo/set-up-a-remote-machine-for-deep-learning-with-jupyter-notebook-1946729f9fc

In [9]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())

True


In [10]:
# this ensures that the current current PyTorch installation was built with MPS activated
print(torch.backends.mps.is_built())

True


In [3]:
os.environ['MASTER_ADDR'] = '0.0.0.0'
os.environ['MASTER_PORT'] = '8888'
os.environ['WORLD_SIZE'] = '1'
os.environ['RANK'] = '0'

In [4]:
# Specify the GPU device ID
device = torch.device("mps")

## Build the Model
Define the architecture of the deep learning model.
- Setting the hidden_size to 128 is a common practice in deep learning. It is a reasonable starting point that strikes a balance between model capacity and computational efficiency.
- In this case, the task is to predict the exercise (activity) name from the given sensor data. Since this is a multi-class classification problem with more than two possible activity labels, the CrossEntropyLoss function is a suitable choice for the loss function.
- Adam (Adaptive Moment Estimation) is a suitable optimizer because of several reasons:
  - In exercise recognition tasks, different activities may have different levels of complexity and variations in their sensor data. The adaptive learning rate of Adam allows the model to dynamically adjust the learning rate for each parameter, leading to faster convergence and potentially better generalization.
  - Adam incorporates momentum, which helps accelerate the training process by allowing the optimizer to accumulate past gradients and move in the relevant directions with more speed. This can be particularly useful in exercise recognition tasks where the model needs to capture temporal dependencies in the sensor data. The momentum term in Adam can assist in better capturing the patterns and dynamics of the exercises.
  - In exercise recognition tasks, it is common for certain sensor data points to have sparse gradients. For example, there might be instances where the sensor data does not change significantly over time for certain activities. Adam's ability to handle sparse gradients makes it a suitable optimizer for such cases, ensuring stable and effective training even when some gradients are infrequent or noisy.

In [9]:
# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long, device=device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long, device=device)

In [10]:
# Create a PyTorch dataset and dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
# Define the deep learning model architecture
class ExerciseClassifier1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ExerciseClassifier1, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
# Set the hyperparameters
input_size = len(X_train)
hidden_size = 128
num_classes = len(df.activity.unique())

# Initialize the model
model1 = ExerciseClassifier1(input_size, hidden_size, num_classes)
model1.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.01)

# Train the deep learning model
num_epochs = 10
for epoch in range(num_epochs):
    model1.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10, Loss: 1.3357
Epoch 2/10, Loss: 1.2267
Epoch 3/10, Loss: 1.2069
Epoch 4/10, Loss: 1.2000
Epoch 5/10, Loss: 1.2018
Epoch 6/10, Loss: 1.1955
Epoch 7/10, Loss: 1.1912
Epoch 8/10, Loss: 1.1916
Epoch 9/10, Loss: 1.1896
Epoch 10/10, Loss: 1.1936


## Evaluate the Model
1. Use the trained model to make predictions on the testing dataset.
2. Evaluate the performance of the model using accuracy as performance metric.

In [17]:
# Evaluate the model
model1.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy1 = correct / total
print('Test Accuracy:', test_accuracy1)

Test Accuracy: 0.6419440529686454


In [18]:
# save the entire model (including architecture and parameters)
torch.save(model1, 'models/model1.pth')